In [1]:
reddit_post_mapping = {
    'CharacterAI': ['CharacterAI', 'CharacterAi_NSFW'],
    'HuggingChat': ['HuggingChat'],
    'OpenAI': ['GPTStore', 'GPTStoreFR', 'ChatGPTStore'],
    'Poe': ['Poe_AI', 'PoeAI_NSFW'],
}

In [4]:
import pandas as pd
import praw
import json

with open('.reddit.json') as f:
    config = json.load(f)
    reddit = praw.Reddit(**config)
    for forum in reddit_post_mapping:
        df = pd.DataFrame()
        for subreddit in reddit_post_mapping[forum]:
            posts = []
            for submission in reddit.subreddit(subreddit).hot(limit=None):
                posts.append({
                    'Title': submission.title,
                    'Question': submission.selftext,
                    'subreddit': submission.subreddit.display_name,
                    'Link': "https://www.reddit.com" + submission.permalink,
                    'Creation Time': submission.created,
                    'Reply Count': submission.num_comments,
                    'Upvote Ratio': submission.upvote_ratio
                })
            df = pd.concat([df, pd.DataFrame(posts)], ignore_index=True)
        df.to_json(f'{forum}_reddit.json', orient='records', indent=4)

In [2]:
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd
import numpy as np
import requests

tag_store_links = []

driver = webdriver.Chrome()
driver.implicitly_wait(5)

url = 'https://community.openai.com/tags'
driver.get(url)

for tag in driver.find_elements(By.XPATH, '//div[@class="tag-box"]/a'):
    tag_name = tag.get_attribute('data-tag-name')
    if 'store' in tag_name.lower():
        tag_store_links.append(tag.get_attribute('href'))
        
post_store_links = set()

for tag_link in tag_store_links:
    driver.get(tag_link)
    for post in driver.find_elements(By.XPATH, '//tbody[@class="topic-list-body"]/tr'):
        post_link = post.find_element(By.XPATH, './/a[@role="heading"]').get_attribute('href')
        post_store_links.add(post_link)
        
posts = pd.DataFrame()
for post_link in post_store_links:
    json_data = requests.get(post_link + '.json').json()
    post = {}
    post['Title'] = json_data['title']
    post['Creation Time'] = json_data['created_at']
    post['View Count'] = json_data['views']
    post['Reply Count'] = json_data['reply_count']
    post['Like Count'] = json_data['like_count']
    post['Link'] = post_link
    post['Tags'] = json_data['tags']
    post['Question'] = json_data['post_stream']['posts'][0]['cooked']
    accpeted_answer = np.nan
    for reply in json_data['post_stream']['posts'][1:]:
        if reply['accepted_answer']:
            accpeted_answer = reply['cooked']
            break
    post['Accepted Answer'] = accpeted_answer
    post = pd.DataFrame([post])
    posts = pd.concat([posts, post], ignore_index=True)
posts.to_json('OpenAI_official_website.json', orient='records', indent=4)

driver.quit()


In [13]:
from selenium.webdriver.common.by import By
from selenium import webdriver
import time

driver = webdriver.Chrome()
driver.implicitly_wait(5)

index = 3268
while True:
    url = f'https://huggingface.co/spaces?p={index}&sort=created'
    driver.get(url)
    space = driver.find_elements(By.XPATH, '//article/a')
    if not space:
        break
    space_urls = [s.get_attribute('href') for s in space]
    index += 1
    print(index)
    time.sleep(10)

    if index % 10 == 0:
        with open('huggingface_spaces.text', 'a') as f:
            for url in space_urls:
                f.write(url + '\n')

with open('huggingface_spaces.text', 'a') as f:
    for url in space_urls:
        f.write(url + '\n')